In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import module
import importlib
try:
    importlib.reload(module) # reload module
except NameError:
    pass

In [10]:
path = Path.cwd().parents[0] / "output/vote_result.xlsx"
df = pd.read_excel(path)

In [11]:
partai_vote_sample = {
    "A": 100_000, 
    "B": 80_000, 
    "C": 50_000, 
    "D": 15_000, 
    "E": 2_000, 
}
result = module.get_selected_partai(partai_vote_sample, 
                                    num_selected=8, 
                                    num_almost_selected=2, 
                                    with_rank=True)
result

[('A', 1),
 ('B', 1),
 ('C', 1),
 ('A', 2),
 ('B', 2),
 ('A', 3),
 ('C', 2),
 ('B', 3),
 ('D', 1),
 ('A', 4)]

In [12]:
for no in list(range(1, 11)):

    dapil_no = no
    num_calon_selected = df.loc[df["dapil_no"] == dapil_no, 'terpilih'].sum()
    num_calon_almost_selected = 10

    partai_vote, calon_vote = module.get_dapil_data(df, dapil_no) 
    selected_partai = module.get_selected_partai(partai_vote, num_calon_selected, num_calon_almost_selected, with_rank=True) 
    selected_calon = module.get_selected_calon(calon_vote, selected_partai, with_partai=True)
    # module.verify_if_selected(df, selected_calon, dapil_no)

    selected_calon.sort(key=lambda x: x[0])
    print(selected_calon)

[('Demokrat', 'Desie Christhyana Sari'), ('Gerindra', 'Iman Satria'), ('Gerindra', 'Dr. Dian Pratama, Sp.OG.'), ('Gerindra', 'H. Nuchbatillah, S.H.'), ('Gerindra', 'H. Fajar Sidik'), ('Golkar', 'Basri Baco, S.E., M.M.'), ('Golkar', 'Fitri Aprinasari Utami, S.E.'), ('Nasdem', 'M. Hariadi Anwar'), ('PAN', 'Riano P. Ahmad'), ('PDIP', 'H. Prasetyo Edi Marsudi, S.H.'), ('PDIP', 'Pandapotan Sinaga, S.E., M.M.'), ('PDIP', 'Wa Ode Herlina'), ('PDIP', 'Slamet Riyadi'), ('PDIP', 'Ellyzabeth CH. Mailoa'), ('PKB', 'H. Mualif, S. Pd.'), ('PKS', 'H. Dany Anwar'), ('PKS', 'H. Ismail, S.Pd.'), ('PKS', 'Israyani, S.P.'), ('PKS', 'H. Agus S.P. Otto, S.H., M.H.'), ('PSI', 'Idris Ahmad, S.K.M.'), ('PSI', 'Johannes Martuah, S.T.'), ('Perindo', 'Pahala Tua Sianturi')]
[('Demokrat', 'Hj. Neneng Hasanah, S.IP., M.M.'), ('Gerindra', 'S. Andyka'), ('Gerindra', 'M. Iqba Maulana'), ('Gerindra', 'Saefudin'), ('Golkar', 'Dimaz Raditya'), ('Nasdem', 'H. Muhammad Idris, S.E.'), ('Nasdem', 'Capt. H. Subandi'), ('PAN',

In [13]:
df.loc[:, ["dapil_no", "dapil_nama"]].drop_duplicates()

,dapil_no,dapil_nama
0,1,Jakarta Pusat
185,2,Jakarta Utara A & Kab. Kep. Seribu
322,3,Jakarta Utara B
456,4,Jakarta Timur A
609,5,Jakarta Timur B
767,6,Jakarta Timur C
923,7,Jakarta Selatan A
1074,8,Jakarta Selatan B
1259,9,Jakarta Barat A
1438,10,Jakarta Barat B
